In [1]:
import pandas as pd
import os
import numpy as np
import pandas as pd
from surprise import AlgoBase
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate
from surprise.model_selection.split import KFold
from surprise import accuracy
from sklearn.metrics import mean_squared_error

In [2]:
class NaiveFilter(AlgoBase):
    def __init__(self):
        AlgoBase.__init__(self)
    
    def fit(self, train):
        AlgoBase.fit(self, train)
        self.naive_predict(train)
        return self
    
    def naive_predict(self, dataset):
        self.dataset = dataset
        self.dict_user_mean = {}
        self.global_mean = None
        for user in self.dataset.ur:
            self.dict_user_mean[user] = np.mean([r for (_, r) in self.dataset.ur[user]])
    
    def estimate(self, u, i):
        if u in self.dict_user_mean:
            return self.dict_user_mean[u]
        else:
            return self.trainset.global_mean

In [15]:
def naive_trim(data, trim_type):
    kf = KFold(n_splits=10)
    acc_rmse = 0
    rmse_prediction = []
    for train, test in kf.split(data):
        test_trim = None
        if trim_type == "pop":
            test_trim = preprocess_pop(test)
        elif trim_type == "unpop":
            test_trim = preprocess_unpop(test)
        elif trim_type == "high_var":
            test_trim = preprocess_high_var(test)
        predictions = naive_algo.test(test_trim)
        #y_true = [test[i][2] for i in range(len(test))]
        #rmse_prediction.append(mean_squared_error(y_true, predictions))
        
        rmse_prediction.append(accuracy.rmse(predictions, verbose=False))
    acc_rmse = np.mean(rmse_prediction)
    print('The average RMSE is -> ' + str(acc_rmse))

In [16]:
file_name = 'ratings.csv'#Please change this path into absolute path to "ratings.csv"
data_raw = pd.read_csv(file_name)
data_table = data_raw.pivot_table(values = 'rating', index = ['userId'], columns = ['movieId'], fill_value=0)
data_mat = pd.DataFrame.as_matrix(data_table)
movieId = list(data_table.columns)
userId = list(data_table.index)

D:\Softwares\Anaconda\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.


In [17]:
freq_movie = np.sum(data_mat!=0, axis=0)
pop_idx = list(np.where(freq_movie > 2))[0]
unpop_idx = list(np.where(freq_movie <= 2))[0]
pop_five_idx = list(np.where(freq_movie >= 5))[0]

In [18]:
pop_movie = []
for i in range(len(pop_idx)):
    pop_movie.append(movieId[pop_idx[i]])
pass

unpop_movie = []
for i in range(len(unpop_idx)):
    unpop_movie.append(movieId[unpop_idx[i]])
pass

data_mat_rm_zero = data_raw.pivot_table(values = 'rating', index = ['userId'], columns = ['movieId'])
movie_var = np.var(data_mat_rm_zero, axis = 0)
high_var = []
for i in range(len(pop_five_idx)):
    movie = pop_five_idx[i]
    if list(movie_var)[movie] >= 2:
        high_var.append(movieId[movie])

print(len(pop_movie))
print(len(unpop_movie))
print(len(high_var)) 

4980
4744
40


In [19]:
def preprocess_pop(testset):
    pop_trim_test = []
    for i in range(len(testset)):
        if int(testset[i][1]) in pop_movie:
            pop_trim_test.append(testset[i])

    return pop_trim_test

def preprocess_unpop(testset):
    unpop_trim_test = []
    for i in range(len(testset)):
        if int(testset[i][1]) in unpop_movie:
            unpop_trim_test.append(testset[i])

    return unpop_trim_test

def preprocess_high_var(testset):
    high_var_trim = []
    for i in range(len(testset)):
        if int(testset[i][1]) in high_var:
            high_var_trim.append(testset[i])
            
    return high_var_trim

In [20]:
file_name = 'ratings.csv'#Please change this path into absolute path to "ratings.csv"
reader = Reader(line_format='user item rating', sep=',', skip_lines=1, rating_scale=(0.0, 5))
data = Dataset.load_from_file(file_name, reader=reader)

In [21]:
naive_algo = NaiveFilter()
train_all = data.build_full_trainset()
naive_algo.fit(train_all)

In [26]:
kf = KFold(n_splits=10)
acc_rmse = 0
rmse_prediction = []
for train, test in kf.split(data):
    predictions = naive_algo.test(test)
    rmse_prediction.append(accuracy.rmse(predictions, verbose=False))
acc_rmse = np.mean(rmse_prediction)
print("The average RMSE is:")
print(acc_rmse)

The average RMSE is:
0.9346883673419637


In [27]:
naive_trim(data, 'pop')

The average RMSE is -> 0.9323220065064985


In [30]:
naive_trim(data, 'unpop')

The average RMSE is -> 0.9705325889814593


In [29]:
naive_trim(data, 'high_var')

The average RMSE is -> 1.478738092572781
